# [Module 2] Monitor and Analyze Training Jobs Using Metrics

Amazon SageMaker 학습 작업은 모델이 학습 데이터셋에서 샘플들을 제시하여 예측하도록 가르치는 반복적인 프로세스입니다.
일반적으로 학습 알고리즘은 학습 오차(training error) 및 예측 정확도(prediction accuracy)와 같은 여러 지표들(metrics)을 계산합니다. 이러한 지표들은 모델이 잘 학습되고 있는지 확인하고 신규 데이터를 잘 예측할 수 있도록 일반화합니다.

학습 알고리즘은 이러한 지표들의 값을 Amazon SageMaker가 모니터링하고 실시간으로 Amazon CloudWatch로 전송하는 로그에 기록합니다.

Amazon SageMaker가 사용자 정의 알고리즘의 로그를 파싱하고 알고리즘이 생성하는 지표를 CloudWatch로 보내려면, 학습 작업을 설정할 때 Amazon SageMaker가 CloudWatch로 전송할 지표를 지정해야 합니다.<br>
이 때, 여러분은 전송하려는 지표들의 이름과 정규표현식들(regular expressions)을 지정해야 합니다. 정규표현식들은 Amazon SageMaker가 해당 지표를 찾기 위해 필요합니다.

본 실습에서는 Amazon SageMaker를 활용한 지표 모니터링에 대해 실시합니다. 실습 전반에서는 Amazon CloudWatch를 사용하여 모니터링을 실시합니다.
실습 후반에서는 Keras에서 출력하는 학습 로그를 활용하여 TensorBoard에서 학습 상황을 모니터링해 봅니다.

## Defining Training Metrics (Amazon SageMaker Python SDK)

Estimator 객체를 초기화할 때 지표 이름 및 정규식 목록을 metric_definitions 인수로 지정하여 CloudWatch로 보내려는 지표를 정의하세요. 예를 들어 CloudWatch에서 train:error 및 validation:error 지표를 모두 모니터링하려는 경우 Estimator 초기화 방법은 다음 코드와 같습니다.

```python 
estimator = Estimator(image_name=ImageName,
            role='SageMakerRole', train_instance_count=1,
            train_instance_type='ml.c4.xlarge',            
            k=10,
            metric_definitions=[
                   {'Name': 'train:error', 'Regex': 'Train_error=(.*?);'},
                   {'Name': 'validation:error', 'Regex': 'Valid_error=(.*?);'}
            ])
```

## Monitoring the CIFAR10 training
아래 작업들을 직접 수행해 보세요.
- SageMaker 콘솔에서 이전에 학습했던 학습 작업(cifar10_xxxxxx)을 찾아 보세요.
- 작업 세부 사항(job details)을 열고 CloudWatch 로그를 확인합니다.
- 로그에 맞는 지표 정규식을 설정하세요. 정규식 도구들(regex tools)을 사용하여 정규식을 확인하고 ()를 사용하여 각 matric을 확인합니다.

In [1]:

metric_definitions = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.* - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.* - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* ([0-9]+)ms/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+'}
]

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [3]:
sagemaker.__version__

'1.72.1'

In [4]:
prefix = 'data/DEMO-cifar10'
dataset_location = os.path.join('s3://', sagemaker_session.default_bucket(), prefix)

`1_Running_TensorFlow_In_SageMaker.ipynb`의 estimator 인스턴스 생성 코드를 그대로 복사 후에
`metric_definitions=metric_definitions` 인자를 추가해 주세요. 

In [5]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-cloudwatch',
                       entry_point='cifar10_keras_tensorboad_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions) # 추가

In [6]:
%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-07 14:55:05 Starting - Starting the training job...
2020-12-07 14:55:08 Starting - Launching requested ML instances......
2020-12-07 14:56:29 Starting - Preparing the instances for training.........
2020-12-07 14:57:46 Downloading - Downloading input data......
2020-12-07 14:58:41 Training - Downloading the training image...
2020-12-07 14:59:20 Training - Training image download completed. Training in progress.2020-12-07 14:59:26,468 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-12-07 14:59:26,966 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperp

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015192/312 [=================>............] - ETA: 9s - loss: 1.9622 - accuracy: 0.2741#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/312 [=================>............] - ETA: 9s - loss: 1.9590 - accuracy: 0.2752#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015  1/312 [..............................] - ETA: 11s - loss: 1.5765 - accuracy: 0.4531#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  3/312 [..............................] - ETA: 11s - loss: 1.5805 - accuracy: 0.4245#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  5/312 [..............................] - ETA: 11s - loss: 1.6112 - accuracy: 0.4109#010#010#010#010#010#010#010#010#010#010#010#010#010#

...

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015195/312 [=================>............] - ETA: 4s - loss: 1.3103 - accuracy: 0.5262#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015197/312 [=================>............] - ETA: 4s - loss: 1.3104 - accuracy: 0.5265#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015195/312 [=================>............] - ETA: 3s - loss: 1.1908 - accuracy: 0.5740#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015197/312 [=================>............] - ETA: 3s - loss: 1.1895 - accuracy: 0.5747#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010


2020-12-07 15:00:59 Uploading - Uploading generated training model#015  1/312 [..............................] - ETA: 11s - loss: 1.2593 - accuracy: 0.5938#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  3/312 [..............................] - ETA: 13s - loss: 1.1949 - accuracy: 0.5625#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  5/312 [..............................] - ETA: 12s - loss: 1.1150 - ac

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015195/312 [=================>............] - ETA: 4s - loss: 1.1050 - accuracy: 0.6085#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015197/312 [=================>............] - ETA: 3s - loss: 1.1039 - accuracy: 0.6088#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010


2020-12-07 15:01:11 Completed - Training job completed
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
[2020-12-07 15:00:53.967 ip-10-0-214-96.us-east-2.compute.internal:25 INFO utils.py:25] The end of training job file will not be written for jobs running under SageMaker.
2020-12-07 15:00:55,061 sagemaker-containers INFO     Reporting training SUCCESS
Training seconds: 205
Billable seconds: 205


### View the job training metrics
SageMaker는 위에서 구성한 정규식을 사용하여 작업 지표(job metrics)를 CloudWatch 지표로 전송했습니다.
이제 SageMaker 콘솔에서 직접 작업 지표을 보실 수 있습니다.

[SageMaker console](https://console.aws.amazon.com/sagemaker/home) 에 로그인하여 최신 학습 작업(latest training job)을 선택하고 모니터 섹션(monitor section)까지 아래로 스크롤하십시오.
CloudWatch 지표를 사용하여 기간(period)을 변경하고 통계치(statistics)들을 설정할 수 있습니다

다음 셀(cell)을 사용하여 지표를 찾아 보세요.

In [7]:
from IPython.core.display import Markdown

link = 'https://console.aws.amazon.com/cloudwatch/home?region='+sagemaker_session.boto_region_name+'#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20'+estimator.latest_training_job.job_name
display(Markdown('CloudWatch metrics: [link]('+link+')'))
display(Markdown('After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)'))

CloudWatch metrics: [link](https://console.aws.amazon.com/cloudwatch/home?region=us-east-2#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20cifar10-cloudwatch-2020-12-07-14-55-04-675)

After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)

## Monitor with TensorBoard

이번에는 TensorBoard로 학습 작업을 실시간으로 모니터링해 봅니다.<br>
TensorBoard는 머신 러닝 실험에 필요한 시각화 및 도구들을 제공합니다.
* Loss 및 accuracy과 같은 metric 추적 및 시각화
* 모델 그래프 (ops 및 layers) 시각화
* 시간 경에 따라 변화하는 가중치(weights), 바이어스(biases) 또는 기타 텐서의 히스토그램 확인
* 저차원 공간으로 임베딩(embedding)
* 이미지, 텍스트 및 오디오 데이터 표시
* 기타 



----
### TODO 1.

먼저, 학습 로그를 TensorBoard로 전송하기 위해 `cifar10_keras_tensorboard_tf2.py` 스크립트를 수정합니다.<br>
Keras에서 TensorBoard를 사용하려면 코드 상단에 `from keras.callbacks import TensorBoard` 구문을 추가해 주세요.

Keras는 디폴트로 각 배치(batch)마다 TensorBoard 로그를 보냅니다. S3에 로그를 전송하면 학습 작업이 느려지기 때문에,
TensorBoard callback을 각 epoch의 끝에서만 로그를 전송할 수 있게 변경해 주는 것이 좋습니다.

----
### TODO 2.

스크립트에 TensorBoard callback을 추가해 주세요 (ModelCheckpoint callback 바로 다음 라인에 아래 줄을 추가해 주세요).
```python
callbacks.append(TensorBoard(log_dir=args.model_output_dir,update_freq='epoch'))
```


### Run a training job with TensorBoard support

In [8]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-tensorboard',
                       entry_point='cifar10_keras_tensorboad_tf2.py',                       
                       source_dir='training_script',
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions)

In [9]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)}, wait=True)
 # Use wait=False to run async jobs

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-07 15:01:50 Starting - Starting the training job...
2020-12-07 15:01:53 Starting - Launching requested ML instances......
2020-12-07 15:02:56 Starting - Preparing the instances for training...............
2020-12-07 15:05:29 Downloading - Downloading input data...
2020-12-07 15:06:05 Training - Downloading the training image......
2020-12-07 15:07:08 Training - Training image download completed. Training in progress..2020-12-07 15:07:14,032 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-12-07 15:07:14,553 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015192/312 [=================>............] - ETA: 8s - loss: 1.9616 - accuracy: 0.2793#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/312 [=================>............] - ETA: 8s - loss: 1.9584 - accuracy: 0.2802#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015  1/312 [..............................] - ETA: 10s - loss: 1.4534 - accuracy: 0.4375#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  3/312 [..............................] - ETA: 10s - loss: 1.5834 - accuracy: 0.4089#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  4/312 [..............................] - ETA: 12s - loss: 1.5773 - accuracy: 0.4141#010#010#010#010#010#010#010#010#010#010#010#010#010#

#015  1/312 [..............................] - ETA: 10s - loss: 1.5069 - accuracy: 0.3984#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  3/312 [..............................] - ETA: 11s - loss: 1.3241 - accuracy: 0.5156#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  5/312 [..............................] - ETA: 10s - loss: 1.3039 - accuracy: 0.5281#010#010#010#010#010#010#010#010#010#010#010#010#010#

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/312 [=================>............] - ETA: 3s - loss: 1.3015 - accuracy: 0.5310#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015196/312 [=================>............] - ETA: 3s - loss: 1.3000 - accuracy: 0.5320#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015198/312 [==================>

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015194/312 [=================>............] - ETA: 3s - loss: 1.1832 - accuracy: 0.5750#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015196/312 [=================>............] - ETA: 3s - loss: 1.1820 - accuracy: 0.5755#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015198/312 [==================>...........] - ETA: 3s -

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015195/312 [=================>............] - ETA: 3s - loss: 1.0898 - accuracy: 0.6122#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015197/312 [=================>............] - ETA: 3s - loss: 1.0900 - accuracy: 0.6119#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015199/312 [==================>...........] - ETA: 3s -


2020-12-07 15:08:53 Uploading - Uploading generated training model
2020-12-07 15:08:53 Completed - Training job completed
Training seconds: 204
Billable seconds: 204


### Install Tensorboard on your local machine

`pip install tensorboard`를 사용하여 [TensorBoard](https://github.com/tensorflow/tensorboard)를 로컬에 설치해 주세요.
S3 로그 디렉토리에 접근하기 위해 TensorBoard 기본 리젼을 설정해 주세요. 여러분은 `AWS_REGION`이라는 환경 변수를 설정하고 환경 변수의 값을 학습 작업이 실행되는 AWS 리전으로 설정하면 됩니다.
예를 들면 `AWS_REGION='us-east-2' tensorboard --logdir model_dir` 입니다.

**여러분은 다음 셀(cell)에서 TensorBoard command를 얻을 수 있습니다.**

여러분은 model_dir(S3 위치)에 접근 하기 위해서는 AccessKey + SecretKey 가 필요 합니다. 
이벤트 엔진을 사용하시는 분은 아래에서 얻을 수 있습니다. AccessKey + SecretKey 를 프로프트 창에 실행 시키시고 아래의 가이드에 따라 진행 해주세요,  <br>
https://dashboard.eventengine.run/dashboard

In [10]:
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
## 만약 트레이닝 완료전에 세션이 끊겼다면 다음과 같이 이전에 실행했던 Job name을 가져와서 다시 세션을 시작합니다.
#estimator = estimator.attach(training_job_name='cifar10-tensorboard-2020-12-07-07-23-40-116')

In [11]:
from IPython.core.display import Markdown
import re
link = 'AWS_REGION=\''+sagemaker_session.boto_region_name+'\' tensorboard --logdir ' + estimator.model_dir + ' --host localhost --port 6006'
link=re.sub('model','output',link)
display(Markdown('본 셀의 output(Tensorboard command) 기억 해주세요.'))
display(Markdown(link))

본 셀의 output(Tensorboard command) 기억 해주세요.

AWS_REGION='us-east-2' tensorboard --logdir s3://sagemaker-us-east-2-870180618679/cifar10-tensorboard-2020-12-07-15-01-49-855/output --host localhost --port 6006

위 command를 복사하여 '!'를 붙인 뒤 아래 셀에서 실행해 줍니다.

In [13]:
!AWS_REGION='us-east-2' tensorboard --logdir s3://sagemaker-us-east-2-870180618679/cifar10-tensorboard-2020-12-07-15-01-49-855/output --host localhost --port 6006

TensorBoard 2.1.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


아래 링크로 정보 확인
```
https://<SageMaker-notebook-instance-name>.notebook.<notebook-region>.sagemaker.aws/proxy/6006/.
```
예) https://startup-sagemaker-workshop.notebook.us-east-2.sagemaker.aws/proxy/6006/

**수고 하셨습니다.**

이제 여러분의 학습 작업을 CloudWatch 지표와 TensorBoard로 확인할 수 있습니다.<br>
다음 노트북으로 계속 진행하기 전에 다른 TensorBoard 설정값들을 [TensorBoard callback configuration](https://keras.io/callbacks/#tensorboard)에서 살펴보세요.